In [1]:
import boto3
import pandas as pd; pd.set_option('display.max_columns', 100)
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import recall_score, classification_report
from sklearn.impute import SimpleImputer, KNNImputer

s3 = boto3.resource('s3')
bucket_name = 'data-445'
bucket = s3.Bucket(bucket_name)

file_key = 'framingham.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

# Reading the data-file
heart = pd.read_csv(file_content_stream)
heart.head()

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [2]:
## Defining input and target
X = heart[['age', 'currentSmoker', 'totChol', 'BMI', 'heartRate']]
Y = heart['TenYearCHD']

## Splitting the data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify = Y)

In [3]:
X_train.describe()

,age,currentSmoker,totChol,BMI,heartRate
count,3390.000000,3390.000000,3349.000000,3374.000000,3389.000000
mean,49.579941,0.492625,236.348761,25.778343,76.053408
std,8.578725,0.500019,43.815666,4.080589,12.080570
min,32.000000,0.000000,107.000000,15.540000,44.000000
25%,42.000000,0.000000,205.000000,23.060000,68.000000
50%,49.000000,0.000000,234.000000,25.380000,75.000000
75%,56.000000,1.000000,263.000000,27.987500,83.000000
max,70.000000,1.000000,464.000000,56.800000,143.000000


In [4]:
X_test.describe()

,age,currentSmoker,totChol,BMI,heartRate
count,848.000000,848.000000,839.000000,845.000000,848.000000
mean,49.604953,0.500000,238.209774,25.896497,75.181604
std,8.550887,0.500295,47.555358,4.079247,11.789678
min,34.000000,0.000000,137.000000,16.480000,45.000000
25%,42.000000,0.000000,206.500000,23.090000,67.000000
50%,49.000000,0.500000,234.000000,25.480000,75.000000
75%,56.000000,1.000000,263.000000,28.250000,82.000000
max,69.000000,1.000000,696.000000,44.270000,120.000000


In [5]:
## Defining the imputer
imputer = KNNImputer(n_neighbors = 5, weights = 'distance').fit(X_train)

## Imputing the missing values
X_train_imp = pd.DataFrame(imputer.transform(X_train), columns = X_train.columns)
X_test_imp = pd.DataFrame(imputer.transform(X_test), columns = X_test.columns)

In [6]:
X_train_imp.describe()

,age,currentSmoker,totChol,BMI,heartRate
count,3390.000000,3390.000000,3390.000000,3390.000000,3390.000000
mean,49.579941,0.492625,236.367096,25.783380,76.052060
std,8.578725,0.500019,43.664845,4.078826,12.079043
min,32.000000,0.000000,107.000000,15.540000,44.000000
25%,42.000000,0.000000,206.000000,23.060000,68.000000
50%,49.000000,0.000000,234.000000,25.380000,75.000000
75%,56.000000,1.000000,263.000000,27.990000,83.000000
max,70.000000,1.000000,464.000000,56.800000,143.000000


In [7]:
X_test_imp.describe()

,age,currentSmoker,totChol,BMI,heartRate
count,848.000000,848.000000,848.000000,848.000000,848.000000
mean,49.604953,0.500000,238.163012,25.888826,75.181604
std,8.550887,0.500295,47.394962,4.074888,11.789678
min,34.000000,0.000000,137.000000,16.480000,45.000000
25%,42.000000,0.000000,207.000000,23.090000,67.000000
50%,49.000000,0.500000,234.000000,25.445000,75.000000
75%,56.000000,1.000000,263.000000,28.242500,82.000000
max,69.000000,1.000000,696.000000,44.270000,120.000000


# Random Forest

In [8]:
RF_md = RandomForestClassifier(n_estimators = 500, max_depth = 5).fit(X_train_imp, Y_train)

## Predicting on test
RF_pred = RF_md.predict_proba(X_test_imp)[:, 1]

## Changing likelihoods to lables
RF_label = np.where(RF_pred < 0.1, 0, 1)

print(classification_report(Y_test, RF_label))

              precision    recall  f1-score   support

           0       0.96      0.38      0.55       719
           1       0.21      0.91      0.34       129

    accuracy                           0.46       848
   macro avg       0.59      0.65      0.44       848
weighted avg       0.85      0.46      0.52       848



# AdaBoost 

In [9]:
Ada_md = AdaBoostClassifier(base_estimator = DecisionTreeClassifier(max_depth = 5), n_estimators = 500, learning_rate = 0.01).fit(X_train_imp, Y_train)

## Predicting on test
Ada_pred = Ada_md.predict_proba(X_test_imp)[:, 1]

## Changing likelihoods to lables
Ada_label = np.where(Ada_pred < 0.1, 0, 1)

print(classification_report(Y_test, Ada_label))

              precision    recall  f1-score   support

           0       0.88      0.10      0.18       719
           1       0.15      0.92      0.27       129

    accuracy                           0.22       848
   macro avg       0.51      0.51      0.22       848
weighted avg       0.77      0.22      0.19       848



In [10]:
# Based on my results (using recall), I would use the random forest model to predict likelihood of coronary disease.